#### Import relevant packages

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# Import NumPy and Pandas
import numpy as np
import pandas as pd

#Import relevant libraries
from skbio.stats.distance import permanova, DistanceMatrix
from skbio.stats.ordination import pcoa
from skbio.stats import subsample_counts
from skbio.stats.composition import multiplicative_replacement, closure, clr
from skbio.stats.distance import pwmantel

from scipy.spatial import procrustes
from scipy.stats import spearmanr

from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.metrics import pairwise_distances, balanced_accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import resample
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.decomposition import PCA

import umap as um

import seaborn as sns

import matplotlib.pyplot as plt

from baycomp import two_on_single

from LANDMark import LANDMarkClassifier
from TreeOrdination import TreeOrdination

from deicode.preprocessing import rclr
from deicode.matrix_completion import MatrixCompletion

from numpy.random import RandomState

from umap import UMAP

from numpy import __version__
print("numpy", __version__)

from pandas import __version__
print("pandas", __version__)

from skbio import __version__
print("skbio", __version__)

from scipy import __version__
print("scipy", __version__)

from sklearn import __version__
print("sklearn", __version__)

from umap import __version__
print("umap", __version__)

from seaborn import __version__
print("seaborn", __version__)

from matplotlib import __version__
print("matplotlib", __version__)

try:
    from baycomp import __version__
    print("baycomp", __version__)
except:
    pass

from deicode import __version__
print("deicode", __version__)

#### Functions for randomization and sub-sampling

In [ ]:
#Function for rarefaction
#https://stackoverflow.com/questions/15507993/quickly-rarefy-a-matrix-in-numpy-python
def rarefaction(M, N, y1, y2, seed=0):
    prng = RandomState(seed) # reproducible results
    noccur = np.sum(M, axis=1) # number of occurrences for each sample
    nvar = M.shape[1] # number of variables
    depth = int(np.percentile(noccur, float(N))) # sampling depth

    rem = np.where(noccur < depth, False, True)
    M_ss = M[rem]
    noccur = noccur[rem]
    
    Mrarefied = np.empty_like(M_ss)
    for i in range(M_ss.shape[0]): # for each sample
        p = M_ss[i] / float(noccur[i]) # relative frequency / probability
        choice = prng.choice(nvar, depth, p=p)
        Mrarefied[i] = np.bincount(choice, minlength=nvar)

    return Mrarefied, y1[rem], y2[rem], rem

#Function for creating random data for use in unsupervised learning
def addcl2(X, y):
    
    X_perm = np.copy(X, "C").transpose()
    for col in range(X_perm.shape[0]):
        X_perm[col] = resample(X_perm[col], replace = False, n_samples = X_perm.shape[1])
        
    y_new = ["Original" for _ in range(X.shape[0])]
    y_new.extend(["Randomized" for _ in range(X.shape[0])])
    y_new = np.asarray(y_new)
    
    X_new = np.vstack((X, X_perm.transpose()))
            
    return X_new, y_new

### Import raw data and process
1) Find and remove rare ASVs (Occurs in 2 or fewer samples)

In [ ]:
#Read in taxa data
taxa_tab = pd.read_csv("Healthy Gut/rdp.out.tmp", delimiter = "\t", header = None).values

#Keep all ASVs assigned to Bacteria and Archaea, remove Cyanobacteria and Chloroplasts
idx = np.where(((taxa_tab[:, 2] == "Bacteria") | (taxa_tab[:, 2] == "Archaea")), True, False)
taxa_tab = taxa_tab[idx]
idx = np.where(taxa_tab[:, 5] != "Cyanobacteria/Chloroplast", True, False)
taxa_tab = taxa_tab[idx]
X_selected = set([x[0] for x in taxa_tab])
taxa_tab_ss = {x[0]: x for x in taxa_tab}

#Read in ASV table
X = pd.read_csv("Healthy Gut/ESV.table", index_col = 0, sep = "\t")
X_col = [entry.split("_")[0] for entry in X.columns.values]
X_features = list(set(X.index.values).intersection(X_selected))
X = X.transpose()[X_features].values

#Convert to presence absence and identify/remove rare ASVs
X_pa = np.where(X > 0, 1, 0)
X_sum = np.sum(X_pa, axis = 0)
X_feat = np.where(X_sum > 2, True, False)
X_features = np.asarray(X_features)[X_feat]
X_orig = np.copy(X[:, X_feat], order = "C")

#Get names of high confidence features
n_list = [4, 7, 10, 13, 16, 19]
X_name = []
cluster_name = []
for row in taxa_tab:
    for entry in X_features:
        if row[0] == entry:
            if float(row[n_list[-1]]) > 0.8:
                X_name.append("%s (%s)" %(row[n_list[-1] - 2], entry))
                cluster_name.append("%s-%s" %(row[n_list[-1] - 2], entry))
                break

            elif float(row[n_list[-2]]) > 0.8:
                X_name.append("%s (%s)" %(row[n_list[-2] - 2], entry))
                cluster_name.append("%s-%s" %(row[n_list[-2] - 2], entry))
                break

            elif float(row[n_list[-3]]) > 0.8:
                X_name.append("%s (%s)" %(row[n_list[-3] - 2], entry))
                cluster_name.append("%s-%s" %(row[n_list[-3] - 2], entry))
                break

            elif float(row[n_list[-4]]) > 0.8:
                X_name.append("%s (%s)" %(row[n_list[-4] - 2], entry))
                cluster_name.append("%s-%s" %(row[n_list[-4] - 2], entry))
                break

            elif float(row[n_list[-5]]) > 0.8:
                X_name.append("%s (%s)" %(row[n_list[-5] - 2], entry))
                cluster_name.append("%s-%s" %(row[n_list[-5] - 2], entry))
                break

            else:
                X_name.append("%s" %entry)
                cluster_name.append("Unclassified-%s" %entry)
                break
                
print(X_orig.shape, len(X_name))

#Read in metadata
y = pd.read_csv("Healthy Gut/metadata.csv", index_col = 0).loc[X_col]
y_subj = y["SUBJECT"].values
y = y["LOCATION"].values


#Correct locations so they are more informative
#P/D = Proximal/Distal
#L/M = Lumen/Mucosa
y = np.where(y == "RS", "Distal Lumen", y)
y = np.where(y == "RB", "Distal Mucosa", y)
y = np.where(y == "LS", "Proximal Lumen", y)
y = np.where(y == "LB", "Proximal Mucosa", y)


#List of phenotypes/datasets to test
pheno = "Proximal Lumen-Proximal Mucosa"
         #"Proximal Lumen-Proximal Mucosa", "Proximal Mucosa-Distal Mucosa", "Proximal Lumen-Distal Lumen"]

pheno_a, pheno_b = pheno.split("-")

idx = np.where(((y == pheno_a) | (y == pheno_b)), True, False)


### Compute balanced accuracy scores and PerMANOVA statistics using 5-fold CV with 3 repeats

In [ ]:
y_sel = y[idx]
y_subj_sel = y_subj[idx]

splitter = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 3, random_state = 0)

scores = []
p_stat = []

for method in ["PA", "CLR", "rCLR"]:
    counter = 0
    if method == "PA":
        metric = "jaccard"
                
    else:
        metric = "euclidean"
    
    print("Method:", method)
    
    #Remove all features with zeros in all samples
    X_transform = X_orig[idx].transpose()
    all_zero = X_transform.sum(axis = 1)
    all_zero = np.where(all_zero > 0, True, False)
    
    X_red = X_orig[idx]
    X_red = X_red[:, all_zero]
    
    if method == "PA":         
        X_rare, y_out, y_subj_ss, _ = rarefaction(X_red, 15, y_sel, y_subj_sel, seed=0)
        
        X_trf = np.where(X_rare > 0, 1, 0)
                       
        M = pcoa(DistanceMatrix(pairwise_distances(X_trf, metric = "jaccard").astype(np.float32)), 
                 number_of_dimensions = 2)
        
        X_proj = M.samples.values
                    
    if method == "CLR":
        X_trf = clr(multiplicative_replacement(closure(X_red)))

        y_subj_ss = y_subj_sel
        y_out = y_sel
        
        M = pcoa(DistanceMatrix(pairwise_distances(X_trf, metric = "euclidean").astype(np.float32)), 
                 number_of_dimensions = 2)
        
        X_proj = M.samples.values             
        
    if method == "rCLR":
        y_subj_ss = y_subj_sel
        y_out = y_sel
        
        A = rclr(X_red.transpose()).transpose()
        M = MatrixCompletion(2, max_iterations = 1000).fit(A)
        X_trf = M.solution
        X_proj = M.U
        
    feature_names = np.asarray(X_name)[all_zero]
    cluster_names = np.asarray(cluster_name)[all_zero]
    
    for train, test in splitter.split(X_trf, y_out):
        counter += 1
        print(counter)

        X_training = X_trf[train]
        X_testing = X_trf[test]
        
        X_train_proj = X_proj[train]
        X_test_proj = X_proj[test]
        
        y_train = y_out[train]
        y_test = y_out[test]
        
        y_train_ss = y_subj_ss[train]
        y_test_ss = y_subj_ss[test]
               
        #TreeOrdination
        clf = TreeOrdination(feature_names = cluster_names, metric = metric).fit(X_training, y_train)    
        class_label = clf.predict(X_testing)
        s = balanced_accuracy_score(y_test, class_label)
        print("TreeOrdination (Default):", s)
        scores.append((method, "TreeOrdination (Default)", s))

        clf_knn = KNeighborsClassifier(weights = "distance", metric = metric).fit(clf.R_final, y_train)
        s = balanced_accuracy_score(y_test, clf_knn.predict(clf.transform(X_testing)))
        print("TreeOrdination (5-KNN):", s)
        scores.append((method, "TreeOrdination (5-KNN)", s))

        clf_knn = KNeighborsClassifier(n_neighbors = 3, weights = "distance", metric = metric).fit(clf.R_final, y_train)
        s = balanced_accuracy_score(y_test, clf_knn.predict(clf.transform(X_testing)))
        print("TreeOrdination (3-KNN):", s)
        scores.append((method, "TreeOrdination (3-KNN)", s))

        clf_lm = LANDMarkClassifier(128, n_jobs = 10).fit(clf.R_final, y_train)
        s = balanced_accuracy_score(y_test, clf_lm.predict(clf.transform(X_testing)))
        print("TreeOrdination (LANDMark):", s)
        scores.append((method, "TreeOrdination (LANDMark)", s))

        clf_dm = DummyClassifier(strategy = "stratified").fit(clf.R_final, y_train)
        s = balanced_accuracy_score(y_test, clf_dm.predict(clf.transform(X_testing)))
        print("TreeOrdination (Random):", s)
        scores.append((method, "TreeOrdination (Random)", s))

        #Raw and Projected
        clf_knn = KNeighborsClassifier(weights = "distance", metric = metric).fit(X_training, y_train)
        s = balanced_accuracy_score(y_test, clf_knn.predict(X_testing))
        print("KNN (Raw):", s)
        scores.append((method, "5-KNN (Raw)", s))

        clf_knn = KNeighborsClassifier(n_neighbors = 3, weights = "distance", metric = metric).fit(X_training, y_train)
        s = balanced_accuracy_score(y_test, clf_knn.predict(X_testing))
        print("3-KNN (Raw):", s)
        scores.append((method, "3-KNN (Raw)", s))

        clf_et = ExtraTreesClassifier(128).fit(X_training, y_train)
        s = balanced_accuracy_score(y_test, clf_et.predict(X_testing))
        print("Extra Trees (Raw):", s)
        scores.append((method, "Extra Trees (Raw)", s))

        clf_lm = LANDMarkClassifier(128, n_jobs = 10).fit(X_training, y_train)
        s = balanced_accuracy_score(y_test, clf_lm.predict(X_testing))
        print("LANDMark (Raw):", s)
        scores.append((method, "LANDMark (Raw)", s))

        clf_dm = DummyClassifier(strategy = "stratified").fit(X_training, y_train)
        s = balanced_accuracy_score(y_test, clf_dm.predict(X_testing))
        print("Random (Raw)):", s)
        scores.append((method, "Random (Raw)", s))
        
        clf_knn = KNeighborsClassifier(weights = "distance", metric = "euclidean").fit(X_train_proj, y_train)
        s = balanced_accuracy_score(y_test, clf_knn.predict(X_test_proj))
        print("KNN (Projection):", s)
        scores.append((method, "5-KNN (Projection)", s))
        
        clf_knn = KNeighborsClassifier(n_neighbors = 3, weights = "distance", metric = "euclidean").fit(X_train_proj, y_train)
        s = balanced_accuracy_score(y_test, clf_knn.predict(X_test_proj))
        print("3-KNN (Projection):", s)
        scores.append((method, "3-KNN (Projection)", s))

        clf_et = ExtraTreesClassifier(128).fit(X_train_proj, y_train)
        s = balanced_accuracy_score(y_test, clf_et.predict(X_test_proj))
        print("Extra Trees (Projection):", s)
        scores.append((method, "Extra Trees (Projection)", s))

        clf_lm = LANDMarkClassifier(128, n_jobs = 10).fit(X_train_proj, y_train)
        s = balanced_accuracy_score(y_test, clf_lm.predict(X_test_proj))
        print("LANDMark (Projection):", s)
        scores.append((method, "LANDMark (Projection)", s))

        clf_dm = DummyClassifier(strategy = "stratified").fit(X_train_proj, y_train)
        s = balanced_accuracy_score(y_test, clf_dm.predict(X_test_proj))
        print("Random (Projection)):", s)
        scores.append((method, "Random (Projection)", s))

        #Get statistics for TreeOrdination - Tree Proximity
        pmanova = permanova(DistanceMatrix(pairwise_distances(clf.R_final, 
                                                              metric = "jaccard").astype(np.float32)), 
                            y_train)

        pseudo_f, pval = pmanova.values[4:6]
        R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
        print("TreeOrdination (Tree Proximity)", pseudo_f, pval, R2)
        p_stat.append((method, "TreeOrdination (Tree Proximity)", pseudo_f, pval, R2))

        #Get statistics for TreeOrdination - Tree Proximity Embedding in Euclidean Space
        pmanova = permanova(DistanceMatrix(pairwise_distances(clf.R_PCA_emb, 
                                                              metric = "euclidean").astype(np.float32)), 
                            y_train)

        pseudo_f, pval = pmanova.values[4:6]
        R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
        print("TreeOrdination", pseudo_f, pval, R2)
        p_stat.append((method, "TreeOrdination (Embedding)", pseudo_f, pval, R2))

        #Get statistics for Transformation
        pmanova = permanova(DistanceMatrix(pairwise_distances(X_train_proj, 
                                                              metric = "euclidean").astype(np.float32)), 
                            y_train)

        pseudo_f, pval = pmanova.values[4:6]
        R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
        print("%s (Projection)" %method, pseudo_f, pval, R2)
        p_stat.append((method, "Projection", pseudo_f, pval, R2))

        #Get statistics for Raw
        manova = permanova(DistanceMatrix(pairwise_distances(X_training, 
                                                                  metric = metric).astype(np.float32)), 
                                y_train)

        pseudo_f, pval = pmanova.values[4:6]
        R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
        print("%s (Raw)" %method, pseudo_f, pval, R2)
        p_stat.append((method, "Raw", pseudo_f, pval, R2))

#Save CSVs
scores = pd.DataFrame(scores, columns = ["Transform", "Model", "Balanced Accuracy Score"])
scores.to_csv("Results/BAS_knn_hg.csv")
    
p_stat = pd.DataFrame(p_stat, columns = ["Transform", "Method", "F-Statistic", "P-value", "R2"])
p_stat.to_csv("Results/PerMANOVA.csv") 

### Plot graphs of Balanced Accuracy Scores, and PerMANOVA F-statistics and P-values

In [ ]:
#Plot Graphs - Balanced Accuracy
df_perm = pd.read_csv("Results/BAS.csv")

g = sns.catplot(x = "Model", y = "Balanced Accuracy Score", 
            hue = "Model", 
            col = "Transform", 
            data = df_perm, 
            kind = "boxen",
            dodge = False)

for axes in g.axes.flat:
    _ = axes.set_xticklabels(axes.get_xticklabels(), rotation=90)

plt.tight_layout()
    
plt.savefig("Results/BAS.svg")
plt.close()

#Plot Graphs - F Statistic
df_perm = pd.read_csv("Results/PerMANOVA.csv")

g = sns.catplot(x = "Method", y = "F-Statistic", 
            hue = "Method", 
            col = "Transform", 
            data = df_perm, 
            kind = "boxen",
            dodge = False)

for axes in g.axes.flat:
    _ = axes.set_xticklabels(axes.get_xticklabels(), rotation=90)

plt.tight_layout()
    
plt.savefig("Results/PerMANOVA_F.svg")
plt.close()

#Plot Graphs - P-value
g = sns.catplot(x = "Method", y = "P-value", 
            hue = "Method", 
            col = "Transform", 
            data = df_perm, 
            kind = "boxen",
            dodge = False)

for axes in g.axes.flat:
    _ = axes.set_xticklabels(axes.get_xticklabels(), rotation=90)

plt.tight_layout()
    
plt.savefig("Results/PerMANOVA_P.svg")
plt.close()

### Pairwise Bayesian T-tests comparing the distribution of balanced accuracy scores and F-statistics

In [ ]:
#Bayesian T-tests
df_perm = pd.read_csv("Results/BAS.csv")

transform_types = ["PA", "rCLR", "CLR"]
model_types = list(set(df_perm["Model"]))

for transform in transform_types:
    data = np.where(df_perm["Transform"] == transform, True, False)
    data = df_perm[data]
        
    final_df = []
        
    for i in range(0, len(model_types) - 1):
        for j in range(i + 1, len(model_types)):
            model_a = model_types[i]
            model_b = model_types[j]
            
            data_a = np.where(data["Model"] == model_a, True, False)
            data_a = data[data_a]["Balanced Accuracy Score"].values
            mu_a = np.mean(data_a)
            std_a = np.std(data_a, ddof = 1)
            
            data_b = np.where(data["Model"] == model_b, True, False)
            data_b = data[data_b]["Balanced Accuracy Score"].values
            mu_b = np.mean(data_b)
            std_b = np.std(data_b, ddof = 1)
            
            p_left, p_rope, p_right = two_on_single(data_a, data_b, rope = 0.05, runs = 3)
            print(transform, "%s-%s" %(model_a, model_b), mu_a, std_a, mu_b, std_b, p_left, p_rope, p_right)
            
            final_df.append((transform, model_a, model_b, mu_a, std_a, mu_b, std_b, p_left, p_rope, p_right))
            
    final_df = pd.DataFrame(final_df, index = None, columns = ["Transform", "Comparison A", "Comparison B", "Mean A", "Std Dev A", 
                                                               "Mean B", "Std Dev B", "Left", "ROPE", "Right"])
    final_df.to_csv("Results/%s_ttest_bas.csv" %transform)
    
df_perm = pd.read_csv("Results/PerMANOVA.csv")

transform_types = ["PA", "rCLR", "CLR"]
model_types = list(set(df_perm["Method"]))

for transform in transform_types:
    data = np.where(df_perm["Transform"] == transform, True, False)
    data = df_perm[data]
        
    final_df = []
        
    for i in range(0, len(model_types) - 1):
        for j in range(i + 1, len(model_types)):
            model_a = model_types[i]
            model_b = model_types[j]
            
            data_a = np.where(data["Method"] == model_a, True, False)
            data_a = data[data_a]["F-Statistic"].values
            mu_a = np.mean(data_a)
            std_a = np.std(data_a, ddof = 1)
            
            data_b = np.where(data["Method"] == model_b, True, False)
            data_b = data[data_b]["F-Statistic"].values
            mu_b = np.mean(data_b)
            std_b = np.std(data_b, ddof = 1)
            
            p_left, p_rope, p_right = two_on_single(data_a, data_b, rope = 0.05, runs = 3)
            print(transform, "%s-%s" %(model_a, model_b), p_left, p_rope, p_right)
            
            final_df.append((transform, model_a, model_b, mu_a, std_a, mu_b, std_b, p_left, p_rope, p_right))
            
    final_df = pd.DataFrame(final_df, index = None, columns = ["Transform", "Comparison A", "Comparison B", "Mean A", "Std Dev A", 
                                                               "Mean B", "Std Dev B", "Left", "ROPE", "Right"])
    final_df.to_csv("Results/%s_ttest_permanova.csv" %transform)


#### Create Graphs

In [ ]:
y_sel = y[idx]
y_subj_sel = y_subj[idx]

#Remove all features with zeros in all samples
X_transform = X_orig[idx].transpose()
all_zero = X_transform.sum(axis = 1)
all_zero = np.where(all_zero > 0, True, False)

X_red = X_orig[idx]
X_red = X_red[:, all_zero]

feature_names = np.asarray(X_name)[all_zero]
cluster_names = np.asarray(cluster_name)[all_zero]

data_index = [i for i in range(X_red.shape[0])]

#Split data into training and testing data
x_idx_train, x_idx_test = train_test_split(data_index,
                                           test_size = 0.2,
                                           random_state = 0,
                                           stratify = y_sel)

#Presence-Absence Graph         
X_rare, y_out, y_subj_ss, ret = rarefaction(X_red, 15, y_sel, y_subj_sel, seed=0)

X_trf = np.where(X_rare > 0, 1, 0)
    
M = pcoa(DistanceMatrix(pairwise_distances(X_trf, metric = "jaccard").astype(np.float32)), 
                       number_of_dimensions = 2)
    
train_ret = []
test_ret = []
counter = 0
for i in range(X_red.shape[0]):
    if ret[i] == True:
        if i in x_idx_train:
            train_ret.append(counter)
            
        elif i in x_idx_test:
            test_ret.append(counter)
            
        counter += 1
        
X_proj = M.samples.values
pc1 = str(M.proportion_explained[0] * 100)[0:5]
pc2 = str(M.proportion_explained[1] * 100)[0:5]
            
X_proj_tr = X_proj[train_ret]
X_proj_te = X_proj[test_ret]

df_pa = np.vstack((X_proj_tr, X_proj_te))
train_test = ["Train Data" for _ in range(len(train_ret))]
train_test.extend(["Test Data" for _ in range(len(test_ret))])
hue_data = np.hstack((y_out[train_ret], y_out[test_ret]))
    
sns.scatterplot(x = df_pa[:, 0],
                y = df_pa[:, 1],
                hue = hue_data,
                style = train_test)

plt.xlabel("PC1 (%s Percent)" %pc1)
plt.ylabel("PC2 (%s Percent)" %pc2)

plt.legend('')
        
plt.savefig("Results/PA.svg")
plt.close()

#CLR Graph
X_trf = clr(multiplicative_replacement(closure(X_red)))

M = PCA(2).fit(X_trf[x_idx_train])
pc1 = str(M.explained_variance_ratio_[0] * 100)[0:5]
pc2 = str(M.explained_variance_ratio_[1] * 100)[0:5]

X_proj_tr = M.transform(X_trf[x_idx_train])
X_proj_te = M.transform(X_trf[x_idx_test])

df_clr = np.vstack((X_proj_tr, X_proj_te))
train_test = ["Train Data" for _ in range(X_proj_tr.shape[0])]
train_test.extend(["Test Data" for _ in range(X_proj_te.shape[0])])
hue_data = np.hstack((y_sel[x_idx_train], y_sel[x_idx_test]))

sns.scatterplot(x = df_clr[:, 0],
                y = df_clr[:, 1],
                hue = hue_data,
                style = train_test)

plt.xlabel("PC1 (%s Percent)" %pc1)
plt.ylabel("PC2 (%s Percent)" %pc2)

subj_dict = {}
subj_org = np.hstack((y_subj_sel[x_idx_train], y_subj_sel[x_idx_test]))
for i, entry in enumerate(subj_org):
    if entry not in subj_dict:
        subj_dict[entry] = []
        
    subj_dict[entry].append(df_clr[i])
        
#for subject, points in subj_dict.items():
 #   if len(points) == 2:
  #      plt.plot([points[0][0], points[1][0]], 
   #              [points[0][1], points[1][1]],
    #             'k-',
     #             alpha = 0.1)

plt.savefig("Results/CLR.svg")
plt.close()

#RPCA        
A = rclr(X_red.transpose()).transpose()
M = MatrixCompletion(2, max_iterations = 1000).fit(A)
X_proj = M.U
pc1 = str(M.explained_variance_ratio[0] * 100)[0:5]
pc2 = str(M.explained_variance_ratio[1] * 100)[0:5]

X_proj_tr = X_proj[x_idx_train]
X_proj_te = X_proj[x_idx_test]

df_rclr = np.vstack((X_proj_tr, X_proj_te))
train_test = ["Train Data" for _ in range(X_proj_tr.shape[0])]
train_test.extend(["Test Data" for _ in range(X_proj_te.shape[0])])
hue_data = np.hstack((y_sel[x_idx_train], y_sel[x_idx_test]))
    
sns.scatterplot(x = df_rclr[:, 0],
                y = df_rclr[:, 1],
                hue = hue_data,
                style = train_test)

plt.xlabel("PC1 (%s Percent)" %pc1)
plt.ylabel("PC2 (%s Percent)" %pc2)

plt.legend('')
        
plt.savefig("Results/RCLR.svg")
plt.close()

#Tree Ordination Graph
X_trf = clr(multiplicative_replacement(closure(X_red)))

clf = TreeOrdination(feature_names = cluster_names).fit(X_trf[x_idx_train], y_sel[x_idx_train])

pc1 = str(clf.R_PCA.explained_variance_ratio_[0] * 100)[0:5]
pc2 = str(clf.R_PCA.explained_variance_ratio_[1] * 100)[0:5]

X_proj_tr = clf.R_PCA_emb
X_proj_te = clf.R_PCA.transform(clf.tree_emb.transform(clf.transform(X_trf[x_idx_test])))

df_clr = np.vstack((X_proj_tr, X_proj_te))
train_test = ["Train Data" for _ in range(X_proj_tr.shape[0])]
train_test.extend(["Test Data" for _ in range(X_proj_te.shape[0])])
hue_data = np.hstack((y_sel[x_idx_train], y_sel[x_idx_test]))

sns.scatterplot(x = df_clr[:, 0],
                y = df_clr[:, 1],
                hue = hue_data,
                style = train_test)

plt.xlabel("PC1 (%s Percent)" %pc1)
plt.ylabel("PC2 (%s Percent)" %pc2)

subj_dict = {}
subj_org = np.hstack((y_subj_sel[x_idx_train], y_subj_sel[x_idx_test]))
for i, entry in enumerate(subj_org):
    if entry not in subj_dict:
        subj_dict[entry] = []
        
    subj_dict[entry].append(df_clr[i])
        
for subject, points in subj_dict.items():
    if len(points) == 2:
        plt.plot([points[0][0], points[1][0]], 
                 [points[0][1], points[1][1]],
                 'k-',
                  alpha = 0.1)

plt.legend('')
        
plt.savefig("Results/TreeOrdination.svg")

#### Identify ASVs associated with each class using SAGE.
##### The code below will train a TreeOrdination model. The model will produce an "unsupervised" projection.
##### Randomization will play a role in which features are among the top

In [ ]:
y_sel = y[idx]
y_subj_sel = y_subj[idx]

#Remove all features with zeros in all samples and transform using CLR
X_transform = X_orig[idx].transpose()
all_zero = X_transform.sum(axis = 1)
all_zero = np.where(all_zero > 0, True, False)

X_red = X_orig[idx]
X_red = X_red[:, all_zero]

feature_names = np.asarray(X_name)[all_zero]
cluster_names = np.asarray(cluster_name)[all_zero]

data_index = [i for i in range(X_red.shape[0])]

X_trf = clr(multiplicative_replacement(closure(X_red)))

#Split data into training and testing data
x_idx_train, x_idx_test = train_test_split(data_index,
                                           test_size = 0.2,
                                           random_state = 0,
                                           stratify = y_sel)

#Set up the classifier
clf = TreeOrdination(n_jobs = 20,
                     feature_names = cluster_names).fit(X_trf[x_idx_train], y_sel[x_idx_train])


In [ ]:
#Plot embeddings of approximate and actual projections

D_tr = clf.l_model.predict(X_trf[x_idx_train])
D_te = clf.l_model.predict(X_trf[x_idx_test])
D_comb = np.vstack((D_tr, D_te))
D_comb_hue = np.hstack((y_sel[x_idx_train], y_sel[x_idx_test]))
D_comb_tr_te = np.hstack((["Train" for _ in range(len(x_idx_train))],
                          ["Test" for _ in range(len(x_idx_test))])
                        )

sns.scatterplot(D_comb[:, 0], D_comb[:, 1], hue = D_comb_hue, style = D_comb_tr_te)
plt.legend(bbox_to_anchor=(1.04, 0.5), loc="center left", borderaxespad=0)
plt.tight_layout()
plt.savefig("Results/Approx_Proj.svg")
plt.close()           

D_tr = clf.R_PCA_emb
D_te = clf.R_PCA.transform(clf.tree_emb.transform(np.hstack([R.proximity(X_trf[x_idx_test]) for R in clf.Rs])))
D_comb = np.vstack((D_tr, D_te))
D_comb_hue = np.hstack((y_sel[x_idx_train], y_sel[x_idx_test]))
D_comb_tr_te = np.hstack((["Train" for _ in range(len(x_idx_train))],
                          ["Test" for _ in range(len(x_idx_test))])
                        )
sns.scatterplot(D_comb[:, 0], D_comb[:, 1], hue = D_comb_hue, style = D_comb_tr_te)
plt.legend(bbox_to_anchor=(1.04, 0.5), loc="center left", borderaxespad=0)
plt.tight_layout()
plt.savefig("Results/Actual_Proj.svg")    
plt.close()

In [ ]:
import sage as sg

I = sg.MarginalImputer(clf.l_model, X_trf[x_idx_train])
Es = sg.SignEstimator(I, "mse")
SageVal = Es(X_trf[x_idx_train], clf.R_PCA_emb, optimize_ordering = False, verbose = True)

SageVal.plot_sign(np.asarray([x for x in cluster_names]), max_features = 50, tick_size = 10, label_size = 10, figsize = (8.5, 11))

plt.savefig("Results/SAGE.svg")



In [ ]:
#Test concordance between test-point projections
D_te = clf.l_model.predict(X_trf)
D_te_f = clf.R_PCA.transform(clf.tree_emb.transform(np.hstack([R.proximity(X_trf
                                                                          ) for R in clf.Rs])))

row_index = [k for k in range(D_te.shape[0])]
                
m_test = procrustes(D_te, 
                            D_te_f)[2]

counts = 0
    
for _ in range(999):
    row_index = np.random.permutation(row_index)

    X_permuted = D_te[row_index]

    m_test_perm = procrustes(X_permuted, D_te_f)[2]

    if m_test_perm <= m_test:
        counts += 1

p_val = (counts + 1)/1000
print("Test of Null Hypothesis:", m_test, p_val)

In [ ]:
#Create waterfall plot showing which ASVs are used to make a  prediction for a test data-point
import shap as sh

Explainer = sh.Explainer(clf.l_model, X_trf[x_idx_train], feature_names = np.asarray([x for x in cluster_names]))
sh_vals = Explainer(X_trf[x_idx_train])

t_idx = [0, 1, 2, 3]

sh.plots.waterfall(sh_vals[t_idx[0], :, 0], max_display = 11)
plt.close()

print(y_sel[x_idx_test][t_idx])

sh.plots.waterfall(sh_vals[t_idx[1], :, 0], max_display = 11)
plt.close()

print(y_sel[x_idx_test][t_idx])

sh.plots.waterfall(sh_vals[t_idx[2], :, 0], max_display = 11)
plt.close()

sh.plots.waterfall(sh_vals[t_idx[3], :, 0], max_display = 11)
plt.close()

print(y_sel[x_idx_test][t_idx])

### Pairwise comparisons between different projection methods

In [ ]:
#Variables to hold data
PMANOVA_all = []
Pro_stat = []

#### Perform Robust Centered Log Ratio, Centered Log Ratio, and Presence-Absence Transformations for Unsupervised Analysis

In [ ]:
#Create the different datasets

y_sel = y[idx]
y_subj_sel = y_subj[idx]

randomized_data_rclr = []
rclr_projs = []
randomized_data_clr = []
randomized_data_pa = []
y_pa = []

for i in range(20):
   
    #Remove all features with zeros in all samples
    X_transform = X_orig[idx].transpose()
    all_zero = X_transform.sum(axis = 1)
    all_zero = np.where(all_zero > 0, True, False)
    
    X_red = X_orig[idx]
    X_red = X_red[:, all_zero]
    
    #Presence-Absence
    X_rare, y_out, y_sel_out, _ = rarefaction(X_red, 15, y_sel, y_sel, seed=0)
    X_trf = np.where(X_rare > 0, 1, 0)
    X_rnd, y_rnd = addcl2(X_rare, y_out)
    randomized_data_pa.append((X_rnd, y_rnd))
    y_pa.append(y_sel_out)

    #CLR
    X_rnd, y_rnd = addcl2(X_red, y_sel)
    X_trf = clr(multiplicative_replacement(closure(X_rnd)))
    randomized_data_clr.append((X_trf, y_rnd))
        
    #rCLR
    X_rnd, y_rnd = addcl2(X_red, y_sel)
    A = rclr(X_rnd.transpose()).transpose()
    M = MatrixCompletion(2, max_iterations = 1000).fit(A)
    X_trf = M.solution
    randomized_data_rclr.append((X_trf, y_rnd))
    
    X_rclr_proj = M.U[0:X_trf.shape[0] // 2]
    rclr_projs.append((X_rclr_proj, y_sel))


#### Robust PCA using rCLR transformed data

In [ ]:
method = "rCLR"

d_size = randomized_data_rclr[i][0].shape[0]

rclr_umap = []
rclr_pcoa = []

for i in range(20):
    #PerMANOVA - Full Distance
    D_ijs = pairwise_distances(randomized_data_rclr[i][0][0:d_size//2], metric = "euclidean")
    pmanova = permanova(DistanceMatrix(D_ijs.astype(np.float32)), rclr_projs[i][1])
    
    pseudo_f, pval = pmanova.values[4:6]
    R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
    print("%s (Full)" %method, pseudo_f, pval, R2)
    PMANOVA_all.append((method, "Full", pseudo_f, pval, R2))
    
    #PerMANOVA - Projection (PCoA)
    D_ijs = pairwise_distances(rclr_projs[i][0], metric = "euclidean")
    pmanova = permanova(DistanceMatrix(D_ijs.astype(np.float32)), rclr_projs[i][1])
    
    pseudo_f, pval = pmanova.values[4:6]
    R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
    print("%s (Projection)" %method, pseudo_f, pval, R2)
    PMANOVA_all.append((method, "Projection", pseudo_f, pval, R2))
        
    #PerMANOVA - Projection (UMAP)
    U_trf = UMAP(n_neighbors = 8,
                 n_components = 2,
                 min_dist = 0.001,
                 metric = "euclidean").fit_transform(randomized_data_rclr[i][0][0:d_size//2])
    
    D_ijs = pairwise_distances(U_trf, metric = "euclidean")
    pmanova = permanova(DistanceMatrix(D_ijs.astype(np.float32)), rclr_projs[i][1])
    
    pseudo_f, pval = pmanova.values[4:6]
    R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
    print("%s (UMAP)" %method, pseudo_f, pval, R2)
    PMANOVA_all.append((method, "UMAP", pseudo_f, pval, R2))
        
    rclr_umap.append(U_trf)
    rclr_pcoa.append(rclr_projs[i][0])
        
#Test concordance between projections (How well each PCoA Projection Compares with a UMAP Projection)
row_index = [k for k in range(rclr_umap[i].shape[0])]
for i in range(0, 20-1):
    for j in range(i+1, 20):
             
        m_test = procrustes(rclr_umap[i], 
                            rclr_pcoa[j])[2]

        counts = 0
    
        for _ in range(999):
            row_index = np.random.permutation(row_index)

            X_permuted = rclr_umap[i][row_index]

            m_test_perm = procrustes(X_permuted, rclr_pcoa[j])[2]

            if m_test_perm <= m_test:
                counts += 1

        p_val = (counts + 1)/1000
        print("Test of Null Hypothesis (PCoA != UMAP):", m_test, p_val)
        
        Pro_stat.append((method, "UMAP-PCoA", m_test, p_val))
        
#Test concordance between projections (How well each PCoA Projection Compares with a PCoA Projection)
row_index = [k for k in range(rclr_umap[i].shape[0])]
for i in range(0, 20-1):
    for j in range(i+1, 20):
             
        m_test = procrustes(rclr_pcoa[i], 
                            rclr_pcoa[j])[2]

        counts = 0
    
        for _ in range(999):
            row_index = np.random.permutation(row_index)

            X_permuted = rclr_pcoa[i][row_index]

            m_test_perm = procrustes(X_permuted, rclr_pcoa[j])[2]

            if m_test_perm <= m_test:
                counts += 1

        p_val = (counts + 1)/1000
        print("Test of Null Hypothesis (PCoA != PCoA):", m_test, p_val)
        
        Pro_stat.append((method, "PCoA-PCoA", m_test, p_val))
     
#Test concordance between projections (How well each UMAP Projection Compares with a UMAP Projection)
row_index = [k for k in range(rclr_umap[i].shape[0])]
for i in range(0, 20-1):
    for j in range(i+1, 20):
             
        m_test = procrustes(rclr_umap[i], 
                            rclr_umap[j])[2]

        counts = 0
    
        for _ in range(999):
            row_index = np.random.permutation(row_index)

            X_permuted = rclr_umap[i][row_index]

            m_test_perm = procrustes(X_permuted, rclr_umap[j])[2]

            if m_test_perm <= m_test:
                counts += 1

        p_val = (counts + 1)/1000
        print("Test of Null Hypothesis (UMAP != UMAP):", m_test, p_val)
        
        Pro_stat.append((method, "UMAP-UMAP", m_test, p_val))


#### Statistics on CLR Transformed data

In [ ]:
method = "CLR"

d_size = randomized_data_clr[i][0].shape[0]

rclr_umap = []
rclr_pcoa = []

for i in range(20):
    #PerMANOVA - Full Distance
    D_ijs = pairwise_distances(randomized_data_clr[i][0][0:d_size//2], metric = "euclidean")
    pmanova = permanova(DistanceMatrix(D_ijs.astype(np.float32)), rclr_projs[i][1])
    
    pseudo_f, pval = pmanova.values[4:6]
    R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
    print("%s (Full)" %method, pseudo_f, pval, R2)
    PMANOVA_all.append((method, "Full", pseudo_f, pval, R2))
    
    #PerMANOVA - Projection (PCoA)
    pcoa_proj = pcoa(pairwise_distances(randomized_data_clr[i][0][0:d_size//2].astype(np.float32), metric = "euclidean"))
    D_ijs = pairwise_distances(pcoa_proj.samples.values[:, [0, 1]], metric = "euclidean")
    pmanova = permanova(DistanceMatrix(D_ijs.astype(np.float32)), rclr_projs[i][1])
    
    pseudo_f, pval = pmanova.values[4:6]
    R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
    print("%s (Projection)" %method, pseudo_f, pval, R2)
    PMANOVA_all.append((method, "Projection", pseudo_f, pval, R2))
        
    #PerMANOVA - Projection (UMAP)
    U_trf = UMAP(n_neighbors = 8,
                 n_components = 2,
                 min_dist = 0.001,
                 metric = "euclidean").fit_transform(randomized_data_clr[i][0][0:d_size//2])
    
    D_ijs = pairwise_distances(U_trf, metric = "euclidean")
    pmanova = permanova(DistanceMatrix(D_ijs.astype(np.float32)), rclr_projs[i][1])
    
    pseudo_f, pval = pmanova.values[4:6]
    R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
    print("%s (UMAP)" %method, pseudo_f, pval, R2)
    PMANOVA_all.append((method, "UMAP", pseudo_f, pval, R2))
        
    rclr_umap.append(U_trf)
    rclr_pcoa.append(pcoa_proj.samples.values[:, [0,1]])
        
#Test concordance between projections (How well each PCoA Projection Compares with a UMAP Projection)
row_index = [k for k in range(rclr_umap[i].shape[0])]
for i in range(0, 20-1):
    for j in range(i+1, 20):
             
        m_test = procrustes(rclr_umap[i], 
                            rclr_pcoa[j])[2]

        counts = 0
    
        for _ in range(999):
            row_index = np.random.permutation(row_index)

            X_permuted = rclr_umap[i][row_index]

            m_test_perm = procrustes(X_permuted, rclr_pcoa[j])[2]

            if m_test_perm <= m_test:
                counts += 1

        p_val = (counts + 1)/1000
        print("Test of Null Hypothesis (PCoA != UMAP):", m_test, p_val)
        
        Pro_stat.append((method, "UMAP-PCoA", m_test, p_val))
        
#Test concordance between projections (How well each PCoA Projection Compares with a PCoA Projection)
row_index = [k for k in range(rclr_umap[i].shape[0])]
for i in range(0, 20-1):
    for j in range(i+1, 20):
             
        m_test = procrustes(rclr_pcoa[i], 
                            rclr_pcoa[j])[2]

        counts = 0
    
        for _ in range(999):
            row_index = np.random.permutation(row_index)

            X_permuted = rclr_pcoa[i][row_index]

            m_test_perm = procrustes(X_permuted, rclr_pcoa[j])[2]

            if m_test_perm <= m_test:
                counts += 1

        p_val = (counts + 1)/1000
        print("Test of Null Hypothesis (PCoA != PCoA):", m_test, p_val)
        
        Pro_stat.append((method, "PCoA-PCoA", m_test, p_val))
     
#Test concordance between projections (How well each UMAP Projection Compares with a UMAP Projection)
row_index = [k for k in range(rclr_umap[i].shape[0])]
for i in range(0, 20-1):
    for j in range(i+1, 20):
             
        m_test = procrustes(rclr_umap[i], 
                            rclr_umap[j])[2]

        counts = 0
    
        for _ in range(999):
            row_index = np.random.permutation(row_index)

            X_permuted = rclr_umap[i][row_index]

            m_test_perm = procrustes(X_permuted, rclr_umap[j])[2]

            if m_test_perm <= m_test:
                counts += 1

        p_val = (counts + 1)/1000
        print("Test of Null Hypothesis (UMAP != UMAP):", m_test, p_val)
        
        Pro_stat.append((method, "UMAP-UMAP", m_test, p_val))

### Statistics on Jaccard distances

In [ ]:
method = "PA"

d_size = randomized_data_pa[i][0].shape[0]

rclr_umap = []
rclr_pcoa = []

for i in range(20):
    #PerMANOVA - Full Distance
    D_ijs = pairwise_distances(randomized_data_pa[i][0][0:d_size//2], metric = "jaccard")
    pmanova = permanova(DistanceMatrix(D_ijs.astype(np.float32)), y_pa[i])
    
    pseudo_f, pval = pmanova.values[4:6]
    R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
    print("%s (Full)" %method, pseudo_f, pval, R2)
    PMANOVA_all.append((method, "Full", pseudo_f, pval, R2))
    
    #PerMANOVA - Projection (PCoA)
    pcoa_proj = pcoa(pairwise_distances(randomized_data_pa[i][0][0:d_size//2].astype(np.float32), metric = "jaccard"))
    D_ijs = pairwise_distances(pcoa_proj.samples.values[:, [0, 1]], metric = "euclidean")
    pmanova = permanova(DistanceMatrix(D_ijs.astype(np.float32)), y_pa[i])
    
    pseudo_f, pval = pmanova.values[4:6]
    R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
    print("%s (Projection)" %method, pseudo_f, pval, R2)
    PMANOVA_all.append((method, "Projection", pseudo_f, pval, R2))
        
    #PerMANOVA - Projection (UMAP)
    U_trf = UMAP(n_neighbors = 8,
                 n_components = 2,
                 min_dist = 0.001,
                 metric = "jaccard").fit_transform(randomized_data_pa[i][0][0:d_size//2])
    
    D_ijs = pairwise_distances(U_trf, metric = "euclidean")
    pmanova = permanova(DistanceMatrix(D_ijs.astype(np.float32)), y_pa[i])
    
    pseudo_f, pval = pmanova.values[4:6]
    R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
    print("%s (UMAP)" %method, pseudo_f, pval, R2)
    PMANOVA_all.append((method, "UMAP", pseudo_f, pval, R2))
        
    rclr_umap.append(U_trf)
    rclr_pcoa.append(pcoa_proj.samples.values[:, [0,1]])
        
#Test concordance between projections (How well each PCoA Projection Compares with a UMAP Projection)
row_index = [k for k in range(rclr_umap[i].shape[0])]
for i in range(0, 20-1):
    for j in range(i+1, 20):
             
        m_test = procrustes(rclr_umap[i], 
                            rclr_pcoa[j])[2]

        counts = 0
    
        for _ in range(999):
            row_index = np.random.permutation(row_index)

            X_permuted = rclr_umap[i][row_index]

            m_test_perm = procrustes(X_permuted, rclr_pcoa[j])[2]

            if m_test_perm <= m_test:
                counts += 1

        p_val = (counts + 1)/1000
        print("Test of Null Hypothesis (PCoA != UMAP):", m_test, p_val)
        
        Pro_stat.append((method, "UMAP-PCoA", m_test, p_val))
        
#Test concordance between projections (How well each PCoA Projection Compares with a PCoA Projection)
row_index = [k for k in range(rclr_umap[i].shape[0])]
for i in range(0, 20-1):
    for j in range(i+1, 20):
             
        m_test = procrustes(rclr_pcoa[i], 
                            rclr_pcoa[j])[2]

        counts = 0
    
        for _ in range(999):
            row_index = np.random.permutation(row_index)

            X_permuted = rclr_pcoa[i][row_index]

            m_test_perm = procrustes(X_permuted, rclr_pcoa[j])[2]

            if m_test_perm <= m_test:
                counts += 1

        p_val = (counts + 1)/1000
        print("Test of Null Hypothesis (PCoA != PCoA):", m_test, p_val)
        
        Pro_stat.append((method, "PCoA-PCoA", m_test, p_val))
     
#Test concordance between projections (How well each UMAP Projection Compares with a UMAP Projection)
row_index = [k for k in range(rclr_umap[i].shape[0])]
for i in range(0, 20-1):
    for j in range(i+1, 20):
             
        m_test = procrustes(rclr_umap[i], 
                            rclr_umap[j])[2]

        counts = 0
    
        for _ in range(999):
            row_index = np.random.permutation(row_index)

            X_permuted = rclr_umap[i][row_index]

            m_test_perm = procrustes(X_permuted, rclr_umap[j])[2]

            if m_test_perm <= m_test:
                counts += 1

        p_val = (counts + 1)/1000
        print("Test of Null Hypothesis (UMAP != UMAP):", m_test, p_val)
        
        Pro_stat.append((method, "UMAP-UMAP", m_test, p_val))

#### Statistics on LANDMark Dissimilarities of rCLR transformed data

In [ ]:
method = "rCLR"

d_size = randomized_data_rclr[i][0].shape[0]

rclr_umap = []
rclr_pcoa = []

for i in range(20):
    X_transformed = LANDMarkClassifier(80, use_nnet = False, n_jobs = 15).fit(randomized_data_rclr[i][0],
                                                                              randomized_data_rclr[i][1]
                                                                             ).proximity(randomized_data_rclr[i][0][0:d_size//2])
    
    S = np.dot(X_transformed, X_transformed.T)
    S = 1 - (S / 80)
    S = np.sqrt(S)
    
    #PerMANOVA - Full Distance
    D_ijs = pairwise_distances(X_transformed, metric = "jaccard")
    pmanova = permanova(DistanceMatrix(D_ijs.astype(np.float32)), rclr_projs[i][1])
    
    pseudo_f, pval = pmanova.values[4:6]
    R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
    print("%s (LANDMark Proximity)" %method, pseudo_f, pval, R2)
    PMANOVA_all.append((method, "LANDMark Proximity", pseudo_f, pval, R2))
    
    #PerMANOVA - Projection (PCoA)
    pcoa_proj = pcoa(pairwise_distances(X_transformed.astype(np.float32), metric = "jaccard"))
    D_ijs = pairwise_distances(pcoa_proj.samples.values[:, [0, 1]], metric = "euclidean")
    pmanova = permanova(DistanceMatrix(D_ijs.astype(np.float32)), rclr_projs[i][1])
    
    pseudo_f, pval = pmanova.values[4:6]
    R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
    print("%s (LANDMark Proximity PCoA Projection)" %method, pseudo_f, pval, R2)
    PMANOVA_all.append((method, "LANDMark Proximity PCoA Projection", pseudo_f, pval, R2))
        
    #PerMANOVA - Projection (UMAP)
    U_trf = UMAP(n_neighbors = 8,
                 n_components = 2,
                 min_dist = 0.001,
                 metric = "jaccard").fit_transform(X_transformed)
    
    D_ijs = pairwise_distances(U_trf, metric = "euclidean")
    pmanova = permanova(DistanceMatrix(D_ijs.astype(np.float32)), rclr_projs[i][1])
    
    pseudo_f, pval = pmanova.values[4:6]
    R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
    print("%s (LANDMark Proximity UMAP Projection)" %method, pseudo_f, pval, R2)
    PMANOVA_all.append((method, "LANDMark Proximity UMAP Projection", pseudo_f, pval, R2))
        
    rclr_umap.append(U_trf)
    rclr_pcoa.append(pcoa_proj.samples.values[:, [0,1]])
        
#Test concordance between projections (How well each PCoA Projection Compares with a UMAP Projection)
row_index = [k for k in range(rclr_umap[i].shape[0])]
for i in range(0, 20-1):
    for j in range(i+1, 20):
             
        m_test = procrustes(rclr_umap[i], 
                            rclr_pcoa[j])[2]

        counts = 0
    
        for _ in range(999):
            row_index = np.random.permutation(row_index)

            X_permuted = rclr_umap[i][row_index]

            m_test_perm = procrustes(X_permuted, rclr_pcoa[j])[2]

            if m_test_perm <= m_test:
                counts += 1

        p_val = (counts + 1)/1000
        print("Test of Null Hypothesis (PCoA != UMAP):", m_test, p_val)
        
        Pro_stat.append((method, "UMAP-PCoA (LANDMark)", m_test, p_val))
        
#Test concordance between projections (How well each PCoA Projection Compares with a PCoA Projection)
row_index = [k for k in range(rclr_umap[i].shape[0])]
for i in range(0, 20-1):
    for j in range(i+1, 20):
             
        m_test = procrustes(rclr_pcoa[i], 
                            rclr_pcoa[j])[2]

        counts = 0
    
        for _ in range(999):
            row_index = np.random.permutation(row_index)

            X_permuted = rclr_pcoa[i][row_index]

            m_test_perm = procrustes(X_permuted, rclr_pcoa[j])[2]

            if m_test_perm <= m_test:
                counts += 1

        p_val = (counts + 1)/1000
        print("Test of Null Hypothesis (PCoA != PCoA):", m_test, p_val)
        
        Pro_stat.append((method, "PCoA-PCoA (LANDMark)", m_test, p_val))
     
#Test concordance between projections (How well each UMAP Projection Compares with a UMAP Projection)
row_index = [k for k in range(rclr_umap[i].shape[0])]
for i in range(0, 20-1):
    for j in range(i+1, 20):
             
        m_test = procrustes(rclr_umap[i], 
                            rclr_umap[j])[2]

        counts = 0
    
        for _ in range(999):
            row_index = np.random.permutation(row_index)

            X_permuted = rclr_umap[i][row_index]

            m_test_perm = procrustes(X_permuted, rclr_umap[j])[2]

            if m_test_perm <= m_test:
                counts += 1

        p_val = (counts + 1)/1000
        print("Test of Null Hypothesis (UMAP != UMAP):", m_test, p_val)
        
        Pro_stat.append((method, "UMAP-UMAP (LANDMark)", m_test, p_val))

#### Statistics on LANDMark Dissimilarities of CLR transformed data

In [ ]:
method = "CLR"

d_size = randomized_data_clr[i][0].shape[0]

rclr_umap = []
rclr_pcoa = []

for i in range(20):
    X_transformed = LANDMarkClassifier(80, use_nnet = False, n_jobs = 15).fit(randomized_data_clr[i][0],
                                                                              randomized_data_clr[i][1]
                                                                             ).proximity(randomized_data_clr[i][0][0:d_size//2])
    
    S = np.dot(X_transformed, X_transformed.T)
    S = 1 - (S / 80)
    S = np.sqrt(S)
    
    #PerMANOVA - Full Distance
    D_ijs = pairwise_distances(X_transformed, metric = "jaccard")
    pmanova = permanova(DistanceMatrix(D_ijs.astype(np.float32)), rclr_projs[i][1])
    
    pseudo_f, pval = pmanova.values[4:6]
    R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
    print("%s (LANDMark Proximity)" %method, pseudo_f, pval, R2)
    PMANOVA_all.append((method, "LANDMark Proximity", pseudo_f, pval, R2))
    
    #PerMANOVA - Projection (PCoA)
    pcoa_proj = pcoa(pairwise_distances(X_transformed.astype(np.float32), metric = "jaccard"))
    D_ijs = pairwise_distances(pcoa_proj.samples.values[:, [0, 1]], metric = "euclidean")
    pmanova = permanova(DistanceMatrix(D_ijs.astype(np.float32)), rclr_projs[i][1])
    
    pseudo_f, pval = pmanova.values[4:6]
    R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
    print("%s (LANDMark Proximity PCoA Projection)" %method, pseudo_f, pval, R2)
    PMANOVA_all.append((method, "LANDMark Proximity PCoA Projection", pseudo_f, pval, R2))
        
    #PerMANOVA - Projection (UMAP)
    U_trf = UMAP(n_neighbors = 8,
                 n_components = 2,
                 min_dist = 0.001,
                 metric = "jaccard").fit_transform(X_transformed)
    
    D_ijs = pairwise_distances(U_trf, metric = "euclidean")
    pmanova = permanova(DistanceMatrix(D_ijs.astype(np.float32)), rclr_projs[i][1])
    
    pseudo_f, pval = pmanova.values[4:6]
    R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
    print("%s (LANDMark Proximity UMAP Projection)" %method, pseudo_f, pval, R2)
    PMANOVA_all.append((method, "LANDMark Proximity UMAP Projection", pseudo_f, pval, R2))
                
    rclr_umap.append(U_trf)
    rclr_pcoa.append(pcoa_proj.samples.values[:, [0,1]])
        
#Test concordance between projections (How well each PCoA Projection Compares with a UMAP Projection)
row_index = [k for k in range(rclr_umap[i].shape[0])]
for i in range(0, 20-1):
    for j in range(i+1, 20):
             
        m_test = procrustes(rclr_umap[i], 
                            rclr_pcoa[j])[2]

        counts = 0
    
        for _ in range(999):
            row_index = np.random.permutation(row_index)

            X_permuted = rclr_umap[i][row_index]

            m_test_perm = procrustes(X_permuted, rclr_pcoa[j])[2]

            if m_test_perm <= m_test:
                counts += 1

        p_val = (counts + 1)/1000
        print("Test of Null Hypothesis (PCoA != UMAP):", m_test, p_val)
        
        Pro_stat.append((method, "UMAP-PCoA (LANDMark)", m_test, p_val))
        
#Test concordance between projections (How well each PCoA Projection Compares with a PCoA Projection)
row_index = [k for k in range(rclr_umap[i].shape[0])]
for i in range(0, 20-1):
    for j in range(i+1, 20):
             
        m_test = procrustes(rclr_pcoa[i], 
                            rclr_pcoa[j])[2]

        counts = 0
    
        for _ in range(999):
            row_index = np.random.permutation(row_index)

            X_permuted = rclr_pcoa[i][row_index]

            m_test_perm = procrustes(X_permuted, rclr_pcoa[j])[2]

            if m_test_perm <= m_test:
                counts += 1

        p_val = (counts + 1)/1000
        print("Test of Null Hypothesis (PCoA != PCoA):", m_test, p_val)
        
        Pro_stat.append((method, "PCoA-PCoA (LANDMark)", m_test, p_val))
     
#Test concordance between projections (How well each UMAP Projection Compares with a UMAP Projection)
row_index = [k for k in range(rclr_umap[i].shape[0])]
for i in range(0, 20-1):
    for j in range(i+1, 20):
             
        m_test = procrustes(rclr_umap[i], 
                            rclr_umap[j])[2]

        counts = 0
    
        for _ in range(999):
            row_index = np.random.permutation(row_index)

            X_permuted = rclr_umap[i][row_index]

            m_test_perm = procrustes(X_permuted, rclr_umap[j])[2]

            if m_test_perm <= m_test:
                counts += 1

        p_val = (counts + 1)/1000
        print("Test of Null Hypothesis (UMAP != UMAP):", m_test, p_val)
        
        Pro_stat.append((method, "UMAP-UMAP (LANDMark)", m_test, p_val))

#### Statistics on LANDMark Dissimilarities of PA transformed data

In [ ]:
method = "PA"

rclr_umap = []
rclr_pcoa = []

for i in range(20):
    X_transformed = LANDMarkClassifier(80, use_nnet = False, n_jobs = 15).fit(randomized_data_pa[i][0],
                                                                              randomized_data_pa[i][1]
                                                                             ).proximity(randomized_data_pa[i][0][0:randomized_data_pa[i][0].shape[0]//2])
    
    S = np.dot(X_transformed, X_transformed.T)
    S = 1 - (S / 80)
    S = np.sqrt(S)
    
    #PerMANOVA - Full Distance
    D_ijs = pairwise_distances(X_transformed, metric = "jaccard")
    pmanova = permanova(DistanceMatrix(D_ijs.astype(np.float32)), y_pa[i])
    
    pseudo_f, pval = pmanova.values[4:6]
    R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
    print("%s (LANDMark Proximity)" %method, pseudo_f, pval, R2)
    PMANOVA_all.append((method, "LANDMark Proximity", pseudo_f, pval, R2))
    
    #PerMANOVA - Projection (PCoA)
    pcoa_proj = pcoa(pairwise_distances(X_transformed.astype(np.float32), metric = "jaccard"))
    D_ijs = pairwise_distances(pcoa_proj.samples.values[:, [0, 1]], metric = "euclidean")
    pmanova = permanova(DistanceMatrix(D_ijs.astype(np.float32)), y_pa[i])
    
    pseudo_f, pval = pmanova.values[4:6]
    R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
    print("%s (LANDMark Proximity PCoA Projection)" %method, pseudo_f, pval, R2)
    PMANOVA_all.append((method, "LANDMark Proximity PCoA Projection", pseudo_f, pval, R2))
        
    #PerMANOVA - Projection (UMAP)
    U_trf = UMAP(n_neighbors = 8,
                 n_components = 2,
                 min_dist = 0.001,
                 metric = "jaccard").fit_transform(X_transformed)
    
    D_ijs = pairwise_distances(U_trf, metric = "euclidean")
    pmanova = permanova(DistanceMatrix(D_ijs.astype(np.float32)), y_pa[i])
    
    pseudo_f, pval = pmanova.values[4:6]
    R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
    print("%s (LANDMark Proximity UMAP Projection)" %method, pseudo_f, pval, R2)
    PMANOVA_all.append((method, "LANDMark Proximity UMAP Projection", pseudo_f, pval, R2))
        
    rclr_umap.append(U_trf)
    rclr_pcoa.append(pcoa_proj.samples.values[:, [0,1]])
        
#Test concordance between projections (How well each PCoA Projection Compares with a UMAP Projection)
row_index = [k for k in range(rclr_umap[i].shape[0])]
for i in range(0, 20-1):
    for j in range(i+1, 20):
             
        m_test = procrustes(rclr_umap[i], 
                            rclr_pcoa[j])[2]

        counts = 0
    
        for _ in range(999):
            row_index = np.random.permutation(row_index)

            X_permuted = rclr_umap[i][row_index]

            m_test_perm = procrustes(X_permuted, rclr_pcoa[j])[2]

            if m_test_perm <= m_test:
                counts += 1

        p_val = (counts + 1)/1000
        print("Test of Null Hypothesis (PCoA != UMAP):", m_test, p_val)
        
        Pro_stat.append((method, "UMAP-PCoA (LANDMark)", m_test, p_val))
        
#Test concordance between projections (How well each PCoA Projection Compares with a PCoA Projection)
row_index = [k for k in range(rclr_umap[i].shape[0])]
for i in range(0, 20-1):
    for j in range(i+1, 20):
             
        m_test = procrustes(rclr_pcoa[i], 
                            rclr_pcoa[j])[2]

        counts = 0
    
        for _ in range(999):
            row_index = np.random.permutation(row_index)

            X_permuted = rclr_pcoa[i][row_index]

            m_test_perm = procrustes(X_permuted, rclr_pcoa[j])[2]

            if m_test_perm <= m_test:
                counts += 1

        p_val = (counts + 1)/1000
        print("Test of Null Hypothesis (PCoA != PCoA):", m_test, p_val)
        
        Pro_stat.append((method, "PCoA-PCoA (LANDMark)", m_test, p_val))
     
#Test concordance between projections (How well each UMAP Projection Compares with a UMAP Projection)
row_index = [k for k in range(rclr_umap[i].shape[0])]
for i in range(0, 20-1):
    for j in range(i+1, 20):
             
        m_test = procrustes(rclr_umap[i], 
                            rclr_umap[j])[2]

        counts = 0
    
        for _ in range(999):
            row_index = np.random.permutation(row_index)

            X_permuted = rclr_umap[i][row_index]

            m_test_perm = procrustes(X_permuted, rclr_umap[j])[2]

            if m_test_perm <= m_test:
                counts += 1

        p_val = (counts + 1)/1000
        print("Test of Null Hypothesis (UMAP != UMAP):", m_test, p_val)
        
        Pro_stat.append((method, "UMAP-UMAP (LANDMark)", m_test, p_val))

#### Statistics on ET Dissimilarities (rCLR)

In [ ]:
method = "rCLR"

rclr_umap = []
rclr_pcoa = []

for i in range(20):
    X_transformed = ExtraTreesClassifier(128).fit(randomized_data_rclr[i][0],
                                                  randomized_data_rclr[i][1]
                                                  ).apply(randomized_data_rclr[i][0][0:randomized_data_rclr[i][0].shape[0]//2])
    
    S = OneHotEncoder(sparse = False).fit_transform(X_transformed)
    S = np.dot(S, S.T)
    S = 1 - (S / 128)
    S = np.sqrt(S)

    #PerMANOVA - Full Distance
    pmanova = permanova(DistanceMatrix(S.astype(np.float32)), rclr_projs[i][1])
    
    pseudo_f, pval = pmanova.values[4:6]
    R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
    print("%s (Extra Trees Proximity)" %method, pseudo_f, pval, R2)
    PMANOVA_all.append((method, "Extra Trees Proximity", pseudo_f, pval, R2))
    
    #PerMANOVA - Projection (PCoA)
    pcoa_proj = pcoa(S.astype(np.float32))
    D_ijs = pairwise_distances(pcoa_proj.samples.values[:, [0, 1]], metric = "euclidean")
    pmanova = permanova(DistanceMatrix(D_ijs.astype(np.float32)), rclr_projs[i][1])
    
    pseudo_f, pval = pmanova.values[4:6]
    R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
    print("%s (Extra Trees Proximity PCoA Projection)" %method, pseudo_f, pval, R2)
    PMANOVA_all.append((method, "Extra Trees Proximity PCoA Projection", pseudo_f, pval, R2))
        
    #PerMANOVA - Projection (UMAP)
    U_trf = UMAP(n_neighbors = 8,
                 n_components = 2,
                 min_dist = 0.001,
                 metric = "precomputed").fit_transform(S)
    
    D_ijs = pairwise_distances(U_trf, metric = "euclidean")
    pmanova = permanova(DistanceMatrix(D_ijs.astype(np.float32)), rclr_projs[i][1])
    
    pseudo_f, pval = pmanova.values[4:6]
    R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
    print("%s (Extra Trees Proximity UMAP Projection)" %method, pseudo_f, pval, R2)
    PMANOVA_all.append((method, "Extra Trees Proximity UMAP Projection", pseudo_f, pval, R2))
        
    rclr_umap.append(U_trf)
    rclr_pcoa.append(pcoa_proj.samples.values[:, [0,1]])
        
#Test concordance between projections (How well each PCoA Projection Compares with a UMAP Projection)
row_index = [k for k in range(rclr_umap[i].shape[0])]
for i in range(0, 20-1):
    for j in range(i+1, 20):
             
        m_test = procrustes(rclr_umap[i], 
                            rclr_pcoa[j])[2]

        counts = 0
    
        for _ in range(999):
            row_index = np.random.permutation(row_index)

            X_permuted = rclr_umap[i][row_index]

            m_test_perm = procrustes(X_permuted, rclr_pcoa[j])[2]

            if m_test_perm <= m_test:
                counts += 1

        p_val = (counts + 1)/1000
        print("Test of Null Hypothesis (PCoA != UMAP):", m_test, p_val)
        
        Pro_stat.append((method, "UMAP-PCoA (Extra Trees)", m_test, p_val))
        
#Test concordance between projections (How well each PCoA Projection Compares with a PCoA Projection)
row_index = [k for k in range(rclr_umap[i].shape[0])]
for i in range(0, 20-1):
    for j in range(i+1, 20):
             
        m_test = procrustes(rclr_pcoa[i], 
                            rclr_pcoa[j])[2]

        counts = 0
    
        for _ in range(999):
            row_index = np.random.permutation(row_index)

            X_permuted = rclr_pcoa[i][row_index]

            m_test_perm = procrustes(X_permuted, rclr_pcoa[j])[2]

            if m_test_perm <= m_test:
                counts += 1

        p_val = (counts + 1)/1000
        print("Test of Null Hypothesis (PCoA != PCoA):", m_test, p_val)
        
        Pro_stat.append((method, "PCoA-PCoA (Extra Trees)", m_test, p_val))
     
#Test concordance between projections (How well each UMAP Projection Compares with a UMAP Projection)
row_index = [k for k in range(rclr_umap[i].shape[0])]
for i in range(0, 20-1):
    for j in range(i+1, 20):
             
        m_test = procrustes(rclr_umap[i], 
                            rclr_umap[j])[2]

        counts = 0
    
        for _ in range(999):
            row_index = np.random.permutation(row_index)

            X_permuted = rclr_umap[i][row_index]

            m_test_perm = procrustes(X_permuted, rclr_umap[j])[2]

            if m_test_perm <= m_test:
                counts += 1

        p_val = (counts + 1)/1000
        print("Test of Null Hypothesis (UMAP != UMAP):", m_test, p_val)
        
        Pro_stat.append((method, "UMAP-UMAP (Extra Trees)", m_test, p_val))

#### Statistics on ET Dissimilarities (CLR)

In [ ]:
method = "CLR"

rclr_umap = []
rclr_pcoa = []

for i in range(20):
    X_transformed = ExtraTreesClassifier(128).fit(randomized_data_clr[i][0],
                                                  randomized_data_clr[i][1]
                                                  ).apply(randomized_data_clr[i][0][0:randomized_data_clr[i][0].shape[0]//2])
    
    S = OneHotEncoder(sparse = False).fit_transform(X_transformed)
    S = np.dot(S, S.T)
    S = 1 - (S / 128)
    S = np.sqrt(S)

    #PerMANOVA - Full Distance
    pmanova = permanova(DistanceMatrix(S.astype(np.float32)), rclr_projs[i][1])
    
    pseudo_f, pval = pmanova.values[4:6]
    R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
    print("%s (Extra Trees Proximity)" %method, pseudo_f, pval, R2)
    PMANOVA_all.append((method, "Extra Trees Proximity", pseudo_f, pval, R2))
    
    #PerMANOVA - Projection (PCoA)
    pcoa_proj = pcoa(S.astype(np.float32))
    D_ijs = pairwise_distances(pcoa_proj.samples.values[:, [0, 1]], metric = "euclidean")
    pmanova = permanova(DistanceMatrix(D_ijs.astype(np.float32)), rclr_projs[i][1])
    
    pseudo_f, pval = pmanova.values[4:6]
    R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
    print("%s (Extra Trees Proximity PCoA Projection)" %method, pseudo_f, pval, R2)
    PMANOVA_all.append((method, "Extra Trees Proximity PCoA Projection", pseudo_f, pval, R2))
        
    #PerMANOVA - Projection (UMAP)
    U_trf = UMAP(n_neighbors = 8,
                 n_components = 2,
                 min_dist = 0.001,
                 metric = "precomputed").fit_transform(S)
    
    D_ijs = pairwise_distances(U_trf, metric = "euclidean")
    pmanova = permanova(DistanceMatrix(D_ijs.astype(np.float32)), rclr_projs[i][1])
    
    pseudo_f, pval = pmanova.values[4:6]
    R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
    print("%s (Extra Trees Proximity UMAP Projection)" %method, pseudo_f, pval, R2)
    PMANOVA_all.append((method, "Extra Trees Proximity UMAP Projection", pseudo_f, pval, R2))
        
    rclr_umap.append(U_trf)
    rclr_pcoa.append(pcoa_proj.samples.values[:, [0,1]])
        
#Test concordance between projections (How well each PCoA Projection Compares with a UMAP Projection)
row_index = [k for k in range(rclr_umap[i].shape[0])]
for i in range(0, 20-1):
    for j in range(i+1, 20):
             
        m_test = procrustes(rclr_umap[i], 
                            rclr_pcoa[j])[2]

        counts = 0
    
        for _ in range(999):
            row_index = np.random.permutation(row_index)

            X_permuted = rclr_umap[i][row_index]

            m_test_perm = procrustes(X_permuted, rclr_pcoa[j])[2]

            if m_test_perm <= m_test:
                counts += 1

        p_val = (counts + 1)/1000
        print("Test of Null Hypothesis (PCoA != UMAP):", m_test, p_val)
        
        Pro_stat.append((method, "UMAP-PCoA (Extra Trees)", m_test, p_val))
        
#Test concordance between projections (How well each PCoA Projection Compares with a PCoA Projection)
row_index = [k for k in range(rclr_umap[i].shape[0])]
for i in range(0, 20-1):
    for j in range(i+1, 20):
             
        m_test = procrustes(rclr_pcoa[i], 
                            rclr_pcoa[j])[2]

        counts = 0
    
        for _ in range(999):
            row_index = np.random.permutation(row_index)

            X_permuted = rclr_pcoa[i][row_index]

            m_test_perm = procrustes(X_permuted, rclr_pcoa[j])[2]

            if m_test_perm <= m_test:
                counts += 1

        p_val = (counts + 1)/1000
        print("Test of Null Hypothesis (PCoA != PCoA):", m_test, p_val)
        
        Pro_stat.append((method, "PCoA-PCoA (Extra Trees)", m_test, p_val))
     
#Test concordance between projections (How well each UMAP Projection Compares with a UMAP Projection)
row_index = [k for k in range(rclr_umap[i].shape[0])]
for i in range(0, 20-1):
    for j in range(i+1, 20):
             
        m_test = procrustes(rclr_umap[i], 
                            rclr_umap[j])[2]

        counts = 0
    
        for _ in range(999):
            row_index = np.random.permutation(row_index)

            X_permuted = rclr_umap[i][row_index]

            m_test_perm = procrustes(X_permuted, rclr_umap[j])[2]

            if m_test_perm <= m_test:
                counts += 1

        p_val = (counts + 1)/1000
        print("Test of Null Hypothesis (UMAP != UMAP):", m_test, p_val)
        
        Pro_stat.append((method, "UMAP-UMAP (Extra Trees)", m_test, p_val))

#### Statistics on ET Dissimilarities (PA)

In [ ]:
method = "PA"

rclr_umap = []
rclr_pcoa = []

for i in range(20):
    X_transformed = ExtraTreesClassifier(128).fit(randomized_data_pa[i][0],
                                                  randomized_data_pa[i][1]
                                                  ).apply(randomized_data_pa[i][0][0:randomized_data_pa[i][0].shape[0]//2])
    
    S = OneHotEncoder(sparse = False).fit_transform(X_transformed)
    S = np.dot(S, S.T)
    S = 1 - (S / 128)
    S = np.sqrt(S)

    #PerMANOVA - Full Distance
    pmanova = permanova(DistanceMatrix(S.astype(np.float32)), y_pa[i])
    
    pseudo_f, pval = pmanova.values[4:6]
    R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
    print("%s (Extra Trees Proximity)" %method, pseudo_f, pval, R2)
    PMANOVA_all.append((method, "Extra Trees Proximity", pseudo_f, pval, R2))
    
    #PerMANOVA - Projection (PCoA)
    pcoa_proj = pcoa(S.astype(np.float32))
    D_ijs = pairwise_distances(pcoa_proj.samples.values[:, [0, 1]], metric = "euclidean")
    pmanova = permanova(DistanceMatrix(D_ijs.astype(np.float32)), y_pa[i])
    
    pseudo_f, pval = pmanova.values[4:6]
    R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
    print("%s (Extra Trees Proximity PCoA Projection)" %method, pseudo_f, pval, R2)
    PMANOVA_all.append((method, "Extra Trees Proximity PCoA Projection", pseudo_f, pval, R2))
        
    #PerMANOVA - Projection (UMAP)
    U_trf = UMAP(n_neighbors = 8,
                 n_components = 2,
                 min_dist = 0.001,
                 metric = "precomputed").fit_transform(S)
    
    D_ijs = pairwise_distances(U_trf, metric = "euclidean")
    pmanova = permanova(DistanceMatrix(D_ijs.astype(np.float32)), y_pa[i])
    
    pseudo_f, pval = pmanova.values[4:6]
    R2 = 1 - 1 / (1 + pmanova.values[4] * pmanova.values[4] / (pmanova.values[2] - pmanova.values[3] - 1))
    print("%s (Extra Trees Proximity UMAP Projection)" %method, pseudo_f, pval, R2)
    PMANOVA_all.append((method, "Extra Trees Proximity UMAP Projection", pseudo_f, pval, R2))
        
    rclr_umap.append(U_trf)
    rclr_pcoa.append(pcoa_proj.samples.values[:, [0,1]])
        
#Test concordance between projections (How well each PCoA Projection Compares with a UMAP Projection)
row_index = [k for k in range(rclr_umap[i].shape[0])]
for i in range(0, 20-1):
    for j in range(i+1, 20):
             
        m_test = procrustes(rclr_umap[i], 
                            rclr_pcoa[j])[2]

        counts = 0
    
        for _ in range(999):
            row_index = np.random.permutation(row_index)

            X_permuted = rclr_umap[i][row_index]

            m_test_perm = procrustes(X_permuted, rclr_pcoa[j])[2]

            if m_test_perm <= m_test:
                counts += 1

        p_val = (counts + 1)/1000
        print("Test of Null Hypothesis (PCoA != UMAP):", m_test, p_val)
        
        Pro_stat.append((method, "UMAP-PCoA (Extra Trees)", m_test, p_val))
        
#Test concordance between projections (How well each PCoA Projection Compares with a PCoA Projection)
row_index = [k for k in range(rclr_umap[i].shape[0])]
for i in range(0, 20-1):
    for j in range(i+1, 20):
             
        m_test = procrustes(rclr_pcoa[i], 
                            rclr_pcoa[j])[2]

        counts = 0
    
        for _ in range(999):
            row_index = np.random.permutation(row_index)

            X_permuted = rclr_pcoa[i][row_index]

            m_test_perm = procrustes(X_permuted, rclr_pcoa[j])[2]

            if m_test_perm <= m_test:
                counts += 1

        p_val = (counts + 1)/1000
        print("Test of Null Hypothesis (PCoA != PCoA):", m_test, p_val)
        
        Pro_stat.append((method, "PCoA-PCoA (Extra Trees)", m_test, p_val))
     
#Test concordance between projections (How well each UMAP Projection Compares with a UMAP Projection)
row_index = [k for k in range(rclr_umap[i].shape[0])]
for i in range(0, 20-1):
    for j in range(i+1, 20):
             
        m_test = procrustes(rclr_umap[i], 
                            rclr_umap[j])[2]

        counts = 0
    
        for _ in range(999):
            row_index = np.random.permutation(row_index)

            X_permuted = rclr_umap[i][row_index]

            m_test_perm = procrustes(X_permuted, rclr_umap[j])[2]

            if m_test_perm <= m_test:
                counts += 1

        p_val = (counts + 1)/1000
        print("Test of Null Hypothesis (UMAP != UMAP):", m_test, p_val)
        
        Pro_stat.append((method, "UMAP-UMAP (Extra Trees)", m_test, p_val))

In [ ]:
Pro_stat_df = pd.DataFrame(Pro_stat, columns = ["Transform", "Comparison", "Statistic", "P-Value"])

#Pro_stat_df = pd.read_csv("Results/ProStat_Metric.csv")

g = sns.catplot(x = "Comparison", y = "Statistic", 
            hue = "Comparison", 
            col = "Transform", 
            data = Pro_stat_df, 
            kind = "boxen",
            dodge = False)

for axes in g.axes.flat:
    _ = axes.set_xticklabels(axes.get_xticklabels(), rotation=90)

plt.tight_layout()
    
plt.savefig("Results/ProStat_metric.svg")
plt.close()

g = sns.catplot(x = "Comparison", y = "P-Value", 
            hue = "Comparison", 
            col = "Transform", 
            data = Pro_stat_df, 
            kind = "boxen",
            dodge = False)

for axes in g.axes.flat:
    _ = axes.set_xticklabels(axes.get_xticklabels(), rotation=90)

plt.tight_layout()
    
plt.savefig("Results/ProStat_p_value.svg")
plt.close()

Pro_stat_df.to_csv("Results/ProStat_Metric.csv")

In [ ]:
PMANOVA_all_Df = pd.DataFrame(PMANOVA_all, columns = ["Transform", "Method", "F-Statistic", "P-Value", "R-Squared"])

#PMANOVA_all_Df = pd.read_csv("Results/DM-DL/FStats_Metric.csv")

g = sns.catplot(x = "Method", y = "F-Statistic", 
            hue = "Method", 
            col = "Transform", 
            data = PMANOVA_all_Df, 
            kind = "boxen",
            dodge = False)

for axes in g.axes.flat:
    _ = axes.set_xticklabels(axes.get_xticklabels(), rotation=90)

plt.tight_layout()
    
plt.savefig("Results/FStats_metric.svg")
plt.close()

g = sns.catplot(x = "Method", y = "P-Value", 
            hue = "Method", 
            col = "Transform", 
            data = PMANOVA_all_Df, 
            kind = "boxen",
            dodge = False)

for axes in g.axes.flat:
    _ = axes.set_xticklabels(axes.get_xticklabels(), rotation=90)

plt.tight_layout()
    
plt.savefig("Results/FStats_p_value.svg")
plt.close()

PMANOVA_all_Df.to_csv("Results/FStats_Metric.csv")

In [ ]:
#Bayesian T-tests
df_perm = pd.read_csv("Results/PL-PM/FStats_Metric.csv")

transform_types = ["rCLR", "CLR", "PA"]
model_types = list(set(df_perm["Method"]))

for transform in transform_types:
    data = np.where(df_perm["Transform"] == transform, True, False)
    data = df_perm[data]

    final_df = []
    for i in range(0, len(model_types) - 1):
        for j in range(i + 1, len(model_types)):
            model_a = model_types[i]
            model_b = model_types[j]
            
            data_a = np.where(data["Method"] == model_a, True, False)
            data_a = data[data_a]["F-Statistic"].values
            mu_a = np.mean(data_a)
            std_a = np.std(data_a, ddof = 1)

            data_b = np.where(data["Method"] == model_b, True, False)
            data_b = data[data_b]["F-Statistic"].values
            mu_b = np.mean(data_b)
            std_b = np.std(data_b, ddof = 1)

                       
            p_left, p_rope, p_right = two_on_single(data_a, data_b, rope = 0.05)
            print(transform, "%s-%s" %(model_a, model_b), p_left, p_rope, p_right)
            
            final_df.append((transform, model_a, model_b, mu_a, std_a, mu_b, std_b, p_left, p_rope, p_right))
            
    final_df = pd.DataFrame(final_df, index = None, columns = ["Transform", "Comparison A", "Comparison B", 
                                                               "Mean A", "Std Dev A", "Mean B", "Std Dev B", 
                                                               "Left", "ROPE", "Right"])
    print(final_df)
    final_df.to_csv("Results/%s_ttest_F-Statistic.csv" %transform)
